In [2]:
from pathlib import Path
import regex as re
from collections import defaultdict
import json, yaml, pickle
import numpy as np
import torch

In [24]:
emissions = np.load('test_emit.npy')
ids = np.load('test_id.npy')
target = np.load('test_target.npy')
hypos = pickle.loads(Path('test_hypos.pkl').read_bytes())
dictionary = [item.split()[0] for item in Path('dict.ltr.txt').read_text().splitlines()]
dictionary = ['<blank>'] + dictionary

In [58]:
e = emissions[-1]
isinf = np.isinf(e[:, 1:]).all(-1)
l = 0
while l < len(isinf) and not isinf[l]: l += 1
e = e[:l]
valid_idxs = np.ones(e.shape[1], dtype=bool)
valid_idxs[1:4] = False
e = e[:, valid_idxs]
e = np.log(np.exp(e) / np.sum(np.exp(e), axis=-1, keepdims=True))

In [99]:
def logsumexp(a: np.ndarray, b: np.ndarray):
    return np.log(np.exp(a) + np.exp(b))

def logsumexpdict(a: dict, b: dict):
    ret = {}
    keys = set(a) | set(b)
    for k in keys:
        ret[k] = logsumexp(a[k], b[k])
    return ret

T, N = e.shape
hs = defaultdict(list)
hs[-1] = [tuple()]
pnb = defaultdict(lambda: defaultdict(lambda: -np.inf))
pb = defaultdict(lambda: defaultdict(lambda: -np.inf))
pb[-1][tuple()] = 0.0
pnb[-1][tuple()] = -np.inf
BLANK_ID = 0
SPACE_ID = 1
for t in range(0, T):
    for h in hs[t-1]:
        last_c = h[-1] if h else None
        for c in range(N):
            cp = e[t, c]
            if c == BLANK_ID:
                pb[t][h] = logsumexp(pb[t][h], logsumexp(pb[t-1][h], pnb[t-1][h]) + cp)
            else:
                if c == last_c:
                    pnb[t][h] = logsumexp(pnb[t][h], pnb[t-1][h] + cp)
                    pnb[t][h + (c,)] = logsumexp(pnb[t][h + (c,)], pb[t-1][h] + cp)
                else:
                    pnb[t][h + (c,)] = logsumexp(pnb[t][h + (c,)], logsumexp(pnb[t-1][h], pb[t-1][h]) + cp)
                
                # 因为后边剪枝了，可能剪掉了前一步已经存在的 h + (c,)。
                if h + (c,) not in hs[t-1]:
                    pb[t][h + (c,)] = logsumexp(pb[t][h + (c,)], logsumexp(pb[t-1][h + (c,)], pnb[t-1][h + (c,)]) + e[t, BLANK_ID])
                    pnb[t][h + (c,)] = logsumexp(pnb[t][h + (c,)], pnb[t-1][h + (c,)] + cp)
                
    hs[t] = [item[0] for item in sorted(logsumexpdict(pb[t], pnb[t]).items(), key=lambda x: x[1], reverse=True)[:20]]
    

/var/folders/h0/xyyjljg92hs8qj6f5zy3f1j80000gn/T/ipykernel_18317/4277480334.py:2: RuntimeWarning: divide by zero encountered in log
  return np.log(np.exp(a) + np.exp(b))


In [100]:
decoded = [''.join(dictionary[i] for i in item) for item in hs[T-1]]
decoded

['THIS|HAPPENED|EVERY|YEAR|AND|THE|YOUNG|FIR|TREE|THAT|HAD|NOW|GROWN|TO|A|VERY|COMELY|SIZES|TREMBLED|AT|THE|SIGHT|FOR|THE|MAGNIFICENT|GREAT|TREES|FELL|TO|THE|EARTH|WITH|NOISE|AND|CRACKING|THE|BRANCHES|WERE|LOPPED|OFF|AND|THE|TREES|LOOKED|LONG|AND|BARE|THEY|WERE|HARDLY|TO|BE|RECOGNIZED|AND|THEN|THEY|WERE|LAIDIN|CARTS|AND|THE|HORSES|DRAGGED|THEM|OUT|OF|THE|WOOD|',
 'THIS|HAPPENED|EVERY|YEAR|AND|THE|YOUNG|FIR|TREE|THAT|HAD|NOW|GROWN|TO|A|VERY|COMELY|SIZES|TREMBLED|AT|THE|SIGHT|FOR|THE|MAGNIFICENT|GREAT|TREES|FELL|TO|THE|EARTH|WITH|NOISE|AND|CRACKING|THE|BRANCHES|WERE|LOPPED|OFF|AND|THE|TREES|LOOKED|LONG|AND|BARE|THEY|WERE|HARDLY|TO|BE|RECOGNIZED|AND|THEN|THEY|WERE|LAID|IN|CARTS|AND|THE|HORSES|DRAGGED|THEM|OUT|OF|THE|WOOD|',
 'THIS|HAPPENED|EVERY|YEAR|AND|THE|YOUNG|FIR|TREE|THAT|HAD|NOW|GROWN|TO|A|VERY|COMELY|SIZES|TREMBLED|AT|THE|SIGHT|FOR|THE|MAGNIFICENT|GREAT|TREES|FELL|TO|THE|EARTH|WITH|NOISE|AND|CRACKING|THE|BRANCHES|WERE|LOPPED|OFF|AND|THE|TREES|LOOKED|LONG|AND|BARE|THEY|WERE|HARDLY|

In [79]:
greedy = torch.unique_consecutive(torch.from_numpy(e.argmax(-1)))
greedy = greedy[greedy != BLANK_ID]
greedy = ''.join(dictionary[i] for i in greedy)
greedy

'THIS|HAPPENED|EVERY|YEAR|AND|THE|YOUNG|FIR|TREE|THAT|HAD|NOW|GROWN|TO|A|VERY|COMELY|SIZES|TREMBLED|AT|THE|SIGHT|FOR|THE|MAGNIFICENT|GREAT|TREES|FELL|TO|THE|EARTH|WITH|NOISE|AND|CRACKING|THE|BRANCHES|WERE|LOPPED|OFF|AND|THE|TREES|LOOKED|LONG|AND|BARE|THEY|WERE|HARDLY|TO|BE|RECOGNIZED|AND|THEN|THEY|WERE|LAIDIN|CARTS|AND|THE|HORSES|DRAGGED|THEM|OUT|OF|THE|WOOD|'

In [101]:
greedy == decoded[0]

True